In [120]:
import pandas as pd
import os

pd.options.display.max_colwidth = 200

def read_data():
    df = pd.DataFrame()
    for f in os.listdir("./data"):
        df = pd.concat([df, pd.read_json(f"./data/{f}")])

    return df.reset_index(drop=True)

df = read_data()
df.describe()

/tmp/ipykernel_56772/193527781.py:14: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,url,date,author,section_1,section_2,url_date,dossier,title,subject,text
count,28091,28091,18926,28091,28091,28091,26641,28068,19485,28091
unique,28090,248,2654,21,463,3762,18189,27120,9464,27783
top,https://www.lapresse.ca/actualites/environnement/la-foret-quebecoise-a-la-croisee-des-chemins/2023-10-30/il-faut-revoir-nos-facons-d-amenager-la-foret-ca-presse.php,2024-03-27 00:00:00,Henri Ouellette-Vézina\n,actualites,hockey,le-monde-en-images.php,le-canadien,\n,\n,
freq,2,3163,363,6698,1598,221,155,222,855,207
first,NaN,2023-02-19 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2024-03-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
from modules.utils import cleanup_dataset

df = cleanup_dataset(df)
df.sort_values("date").loc[14285]

/tmp/ipykernel_56772/1679217179.py:11: FutureWarning:

The default value of regex will change from True to False in a future version.



url                                                                                                                                     https://www.lapresse.ca/cinema/2023-10-28/1995/le-plus-drole-de-la-serie.php
date                                                                                                                                                                                             2024-03-27 00:00:00
author                                                                                                                                                                                               Silvia Galipeau
section_1                                                                                                                                                                                                     cinema
section_2                                                                                                                                           

In [132]:

def cleanup_date_field(df):

    df["url_date"] = pd.to_datetime(df["url_date"], format="%Y-%m-%d", errors="coerce")
    df["url_date"] = df["url_date"].fillna(pd.to_datetime(df["section_2"], format="%Y-%m-%d", errors="coerce"))
    df["url_date"] = df["url_date"].fillna(df["date"])
    df["date"] = df["url_date"]

    # Fix manuel deux articles weird
    df.loc[14285, "date"] = df.loc[14285, "section_2"]
    df = df.drop(index=21114)

    return df

df.sort_values("date").head()

,url,date,author,section_1,section_2,url_date,dossier,title,subject,text,abstract
6880,https://www.lapresse.ca/arts/spectacles/2023-06-21/des-spectacles-pour-celebrer-la-fete-nationale.php,2023-06-21,Véronique Larocque,arts,spectacles,2023-06-21,des-spectacles-pour-celebrer-la-fete-nationale.php,Des spectacles pour célébrer la fête nationale,None,"De Montréal à Saguenay, en passant par Laval, Longueuil, Sherbrooke et Trois-Rivières, voici un aperçu des festivités organisées un peu partout en province en ce 24 juin.",arts\nspectacles\n\nVéronique Larocque\nDes spectacles pour célébrer la fête nationale
6809,https://www.lapresse.ca/sports/hockey/2023-06-22/lnh/la-fin-des-uniformes-de-la-fierte-des-mecontents-se-manifestent.php,2023-06-22,stephen whyno,sports,hockey,2023-06-22,lnh,"La fin des uniformes de la Fierté, des mécontents se manifestent",LNH,Les équipes de la LNH n’auront plus d’uniformes spéciaux pour les échauffements d’avant-match lors des soirées thématiques la saison prochaine. C’est la conclusion retenue par la LNH à la suite du...,"sports\nhockey\nLNH\nstephen whyno\nLa fin des uniformes de la Fierté, des mécontents se manifestent"
6797,https://www.lapresse.ca/actualites/justice-et-faits-divers/2023-06-23/saint-charles-sur-richelieu/une-femme-perit-dans-un-accident-de-moto.php,2023-06-23,None,actualites,justice-et-faits-divers,2023-06-23,saint-charles-sur-richelieu,Une femme périt dans un accident de moto,Saint-Charles-sur-Richelieu,"(Saint-Charles-Sur-Richelieu) Une femme âgée de 52 ans a perdu la vie à la suite d’un accident de moto survenu à Saint-Charles-sur-Richelieu, en Montérégie, jeudi soir. \nUne collision entre la mo...",actualites\njustice-et-faits-divers\nSaint-Charles-sur-Richelieu\n\nUne femme périt dans un accident de moto
6798,https://www.lapresse.ca/actualites/insolite/2023-06-23/belgique/des-sculpteurs-de-sable-creent-un-monde-de-conte-de-fees.php,2023-06-23,None,actualites,insolite,2023-06-23,belgique,Des sculpteurs de sable créent un monde de conte de fées,Belgique,Vingt-cinq sculpteurs de sable venus de toute l'Europe passent 10 jours à créer un monde de conte de fées sur la plage belge de Middelkerke en vue du festival annuel qui ouvrira au public le 1er j...,actualites\ninsolite\nBelgique\n\nDes sculpteurs de sable créent un monde de conte de fées
6799,https://www.lapresse.ca/arts/musique/2023-06-23/gros-programme-pour-petit-piknic.php,2023-06-23,Silvia Galipeau,arts,musique,2023-06-23,gros-programme-pour-petit-piknic.php,Gros programme pour Petit Piknic,None,"Pour ses 20 ans, Piknic Électronik revoit en grande son offre familiale, avec une programmation unique et surtout originale, sur quatre dimanches choisis. \nAvis aux familles qui se cherchent des ...",arts\nmusique\n\nSilvia Galipeau\nGros programme pour Petit Piknic


In [123]:
import plotly.express as px

px.histogram(df["url_date"], title="Répartition des articles récoltés par date et par section", color=df["section_1"], range_x=["2023-06-01", "2024-04-01"])


In [148]:
px.bar(df[(df["section_2"] == "chroniques") & (df["section_1"] == "actualites")]["author"].value_counts().head(10), title="Top 10 des auteurs de chroniques", labels={"value": "Nombre d'articles", "index": "Auteur"})

In [139]:
df["author"].value_counts().head(20)

Henri Ouellette-Vézina        363
Philippe Teisceira-Lessard    255
Guillaume Lefrançois          236
Simon-Olivier Lorange         216
Lila Dussault                 209
Nicholas Richard              205
Vincent Larin                 185
Véronique Larocque            173
Katherine Harvey-Pinard       157
Alexandre Vigneault           156
Jean Siag                     154
Julien Arsenault              154
Louis-Samuel Perron           152
Serge Chapleau                146
Richard Dufour                140
Catherine Handfield           139
Mayssa Ferah                  139
Mathias Brunet                138
Lia Lévesque                  138
Jean-François Téotonio        138
Name: author, dtype: int64

In [141]:
# Répartition des articles récoltés par section
df["section_2"].value_counts()

hockey                                            1598
chroniques                                        1480
europe                                            1437
justice-et-faits-divers                           1279
etats-unis                                        1131
                                                  ... 
des-traitements-qui-pourraient-sauver-des-vies       1
les-rebelles-ont-ils-leur-place-en-politique         1
quand-le-quebec-vise-la-stratosphere                 1
un-modele-qui-fait-ses-preuves                       1
du-motel-a-la-rue                                    1
Name: section_2, Length: 463, dtype: int64

In [146]:
chroniques_df = df[df["section_2"] == "chroniques"]
chroniques_df["section_1"].value_counts()

actualites       416
dialogue         290
arts             240
affaires         210
sports            84
international     75
cinema            52
societe           40
contexte          39
debats            22
gourmand           5
maison             5
voyage             2
Name: section_1, dtype: int64

In [125]:
# python -m spacy download fr_core_news_lg
import spacy

nlp = spacy.load("fr_core_news_lg")

In [126]:
# article = df.loc[10]["text"]
for x in df.loc[1]["text"]:
    article = x
doc = nlp(article)

## Premier essai : Sentiment Analysis

TODO :
- Isoler les articles mentionnant Justin Trudeau
- Produire un score de sentiment par article

In [127]:
trudeau_df = df.loc[df["text"].str.contains("Trudeau")]
art = trudeau_df["text"].iloc[0]
doc = nlp(art)

doc = [x for x in doc.sents if "Trudeau" in x.text]
doc

[La ministre a dit avoir eu une première rencontre « positive et constructive » avec les représentants de Meta jeudi, même si les deux parties restent bien campées sur leurs positions. 
 Durant la rencontre, Mme Ste-Onge a affirmé que le gouvernement Trudeau n’avait guère l’intention de modifier la 
 Loi sur les nouvelles en ligne
 .,
 En signe de protestation, Meta a décidé au début du mois d’août de bloquer l’accès aux nouvelles canadiennes sur Facebook et Instagram, soulevant la colère du gouvernement Trudeau. ,
 Cachant mal sa frustration, le premier ministre Justin Trudeau a accusé les dirigeants de Meta de mettre « leurs profits, les profits de leur corporation, en priorité plutôt que le bien-être et l’information pour les Canadiens ».]

In [128]:
# Exemple de visualisation de NER avec DisplaCy
spacy.displacy.render(doc, style="ent", jupyter=True)

## Google Cloud NLP API

taken from: https://cloud.google.com/natural-language/docs/analyzing-entity-sentiment


la V1 du service a un service "analyze_entity_sentiment" mais seulement pour l'anglais. La V2 a complètement drop la fonctionnalité. Possiblement pcq la tâche est trop difficile?

In [129]:
!pip install --upgrade google-cloud-language

In [130]:
from google.cloud import language_v1


def sample_analyze_entity_sentiment(text_content):
    """
    Analyzing Entity Sentiment in a String

    Args:
      text_content The text content to analyze
    """

    client = language_v1.LanguageServiceClient()

    # text_content = 'Grapes are good. Bananas are bad.'

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.types.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "fr"
    document = {"content": text_content, "type_": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8

    response = client.analyze_entity_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )
    # Loop through entitites returned from the API
    for entity in response.entities:
        print(f"Representative name for the entity: {entity.name}")
        # Get entity type, e.g. PERSON, LOCATION, ADDRESS, NUMBER, et al
        print(f"Entity type: {language_v1.Entity.Type(entity.type_).name}")
        # Get the salience score associated with the entity in the [0, 1.0] range
        print(f"Salience score: {entity.salience}")
        # Get the aggregate sentiment expressed for this entity in the provided document.
        sentiment = entity.sentiment
        print(f"Entity sentiment score: {sentiment.score}")
        print(f"Entity sentiment magnitude: {sentiment.magnitude}")
        # Loop over the metadata associated with entity. For many known entities,
        # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid).
        # Some entity types may have additional metadata, e.g. ADDRESS entities
        # may have metadata for the address street_name, postal_code, et al.
        for metadata_name, metadata_value in entity.metadata.items():
            print(f"{metadata_name} = {metadata_value}")

        # Loop over the mentions of this entity in the input document.
        # The API currently supports proper noun mentions.
        for mention in entity.mentions:
            print(f"Mention text: {mention.text.content}")
            # Get the mention type, e.g. PROPER for proper noun
            print(
                "Mention type: {}".format(
                    language_v1.EntityMention.Type(mention.type_).name
                )
            )

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(f"Language of the text: {response.language}")


In [131]:
sample_analyze_entity_sentiment(trudeau_df["text"].iloc[7])

InvalidArgument: 400 The language fr is not supported for entity_sentiment analysis.

## Hugging Face Libraries

le pipeline "sentiment-analysis" permet de faire du sentiment classification sur un texte out-of-the-box, mais impossible de discriminer les sentiments selon les entités incluent dans le texte.

Ressource intéressante : https://huggingface.co/blog/sentiment-analysis-python

pre-trained models list : https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I loved Star Wars so much!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.999840259552002}]